In [2]:
# %logstop
# %logstart -rtq ~/.logs/ml.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [4]:
# from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [6]:
# %%bash
# mkdir data
# wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
# wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [7]:
import numpy as np
import pandas as pd

In [8]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

Variable                    Label              Description  \
0   PROVNUM  Federal Provider Number  Federal Provider Number   
1  PROVNAME            Provider Name            Provider Name   
2   ADDRESS         Provider Address         Provider Address   
3      CITY            Provider City            Provider City   
4     STATE           Provider State           Provider State   

                            Format  
0        6 alphanumeric characters  
1                             text  
2                             text  
3                             text  
4  2-character postal abbreviation

In [7]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')
data.head()

PROVNUM                                     PROVNAME  \
0  015010                COOSA VALLEY NURSING FACILITY   
1  015012                   HIGHLANDS HEALTH AND REHAB   
2  015014  EASTVIEW REHABILITATION & HEALTHCARE CENTER   
3  015015                PLANTATION MANOR NURSING HOME   
4  015016         ATHENS HEALTH AND REHABILITATION LLC   

                                    ADDRESS        CITY STATE    ZIP  \
0                   315 WEST HICKORY STREET   SYLACAUGA    AL  35150   
1                       380 WOODS COVE ROAD  SCOTTSBORO    AL  35768   
2                  7755 FOURTH AVENUE SOUTH  BIRMINGHAM    AL  35206   
3  6450 OLD TUSCALOOSA HIGHWAY   P O BOX 97    MC CALLA    AL  35111   
4                    611 WEST MARKET STREET      ATHENS    AL  35611   

        PHONE  COUNTY_SSA COUNTY_NAME  BEDCERT  ...  CYCLE_1_NFROMCOMP  \
0  2562495604         600   Talladega       85  ...                  0   
1  2562183708         350     Jackson       50  ...                  0   
2  2058330146         360   Jefferson       92  ...                  0   
3  2054776161         360   Jefferson      103  ...                  0   
4  2562321620         410   Limestone      149  ...                  0   

   CYCLE_1_DEFS_SCORE  CYCLE_1_NUMREVIS  CYCLE_1_REVISIT_SCORE  \
0                  36                 1                      0   
1                  44                 1                      0   
2                  40                 1                      0   
3                  16                 1                      0   
4                  20                 1                      0   

   CYCLE_1_TOTAL_SCORE  CYCLE_1_SURVEY_DATE  CYCLE_2_SURVEY_DATE  FINE_CNT  \
0                   36           2017-04-06           2016-05-26         1   
1                   44           2017-03-16           2016-02-04         0   
2                   40           2016-10-20           2015-12-30         0   
3                   16           2017-03-09           2016-02-11         0   
4                   20           2017-06-01           2016-05-12         0   

  FINE_TOT CYCLE_2_TOTAL_SCORE  
0    15259                  16  
1        0                  12  
2        0                  24  
3        0                  24  
4        0                  28  

[5 rows x 32 columns]

In [8]:
fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')


In [9]:
fine_totals

0        15259
1            0
2            0
3            0
4            0
         ...  
13887        0
13888        0
13889        0
13890        0
13891    70865
Name: FINE_TOT, Length: 13892, dtype: int64

## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [10]:
# fine_totals.groupby(data['STATE']).mean().to_dict()

In [11]:
# estimates = [group_averages[row] for row in data['STATE']]

In [12]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.global_avg = 0

    def fit(self, X, y):
        # Use self.group_averages to store the average penalty by group
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        self.global_avg = y.mean()
        return self

    def predict(self, X):
        # Return a list of predicted penalties based on group of samples in X
#         print (type(X))
        if not isinstance (X, pd.DataFrame):
            X = pd.DataFrame(X)
        return [self.group_averages.get(row, self.global_avg) 
                for row in X[self.grouper]]

In [13]:
gme = GroupMeanEstimator(grouper='STATE')
gme.fit(data, fine_totals)
gme.predict(data)[:5]

[13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515,
 13672.320388349515]

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [14]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [15]:
state_model.predict(data.sample(5))

[18147.119744058502,
 37722.919117647056,
 6634.197226502311,
 8054.977611940299,
 6634.197226502311]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [16]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [17]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [18]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer())
])

simple_features.fit(data, fine_counts > 0);

In [19]:
len(simple_cols)
type(simple_features.fit_transform(data))

numpy.ndarray

In [20]:
pd.DataFrame(data[simple_cols]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 9 columns):
BEDCERT             13892 non-null int64
RESTOT              13483 non-null float64
INHOSP              13892 non-null bool
CCRC_FACIL          13892 non-null bool
SFF                 13892 non-null bool
CHOW_LAST_12MOS     13892 non-null bool
SPRINKLER_STATUS    13892 non-null bool
EXP_TOTAL           13104 non-null float64
ADJ_TOTAL           13104 non-null float64
dtypes: bool(5), float64(3), int64(1)
memory usage: 502.1 KB


In [21]:
pd.DataFrame(simple_features.fit_transform(data)).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13892 entries, 0 to 13891
Data columns (total 9 columns):
0    13892 non-null float64
1    13892 non-null float64
2    13892 non-null float64
3    13892 non-null float64
4    13892 non-null float64
5    13892 non-null float64
6    13892 non-null float64
7    13892 non-null float64
8    13892 non-null float64
dtypes: float64(9)
memory usage: 976.9 KB


**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [22]:
assert data['RESTOT'].isnull().sum() > 0

In [23]:
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV


simple_features_model = Pipeline([
    ('simple', simple_features),
    ('predictor', LogisticRegression())   # add your estimator here
])

In [43]:
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer()),
    ('predictor', RandomForestClassifier()) 
])

simple_features.fit(data, fine_counts > 0)

param_grid = {'imputer__strategy':['mean', 'median'],
              'predictor__n_estimators': range(50, 111, 200), 
              'predictor__max_depth': range(4, 11, 2)}

sfm_gs = GridSearchCV(simple_features,
                      param_grid,
                      n_jobs=-1,
                      cv=5,
                      scoring='accuracy',
                      verbose=1)

sfm_gs.fit(data, fine_counts > 0);

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.8s finished


In [44]:
simple_features_model.fit(data, fine_counts > 0);

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [45]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(sfm_gs))

Your score:  0.9755240355229559


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [46]:
# data.OWNERSHIP.reset_index().isnull().sum()
# data.CERTIFICATION.value_counts()

In [69]:
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('ohe', OneHotEncoder(categories='auto', sparse=False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('ohe', OneHotEncoder(categories='auto' , sparse=False))
])

categorical_features = FeatureUnion([
    ('ownership', owner_onehot),
    ('certification', cert_onehot)
])

categorical_features.fit_transform(data)

array([[1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [48]:
# pd.DataFrame(cert_onehot.fit_transform(data))

In [70]:
pd.DataFrame(cert_onehot.fit_transform(data), 
             columns=cert_onehot.named_steps['ohe'].categories_[0]).head()

Medicaid  Medicare  Medicare and Medicaid
0       0.0       0.0                    1.0
1       0.0       0.0                    1.0
2       0.0       0.0                    1.0
3       0.0       0.0                    1.0
4       0.0       0.0                    1.0

In [71]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [72]:
categorical_features_model = Pipeline([
    ('categorical', categorical_features),
#     ('predictor', RandomForestClassifier()),
    ('predictor', MultinomialNB(alpha=0.01))

])

In [73]:
categorical_features_model.fit(data, fine_counts > 0);

In [74]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  0.9747736478437008


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [75]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression

In [77]:
business_features = FeatureUnion([
    ('simple', simple_features),
#     ('simple', sfm_gs),
    ('categorical', categorical_features)
])

business_features.fit_transform(data)

TypeError: All estimators should implement fit and transform. 'Pipeline(memory=None,
         steps=[('cst',
                 ColumnSelectTransformer(columns=['BEDCERT', 'RESTOT', 'INHOSP',
                                                  'CCRC_FACIL', 'SFF',
                                                  'CHOW_LAST_12MOS',
                                                  'SPRINKLER_STATUS',
                                                  'EXP_TOTAL', 'ADJ_TOTAL'])),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('predictor',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,


In [68]:

business_model = Pipeline([
    ('features', business_features),
    ('poly', PolynomialFeatures(2)),
    ('predictor', LogisticRegression())
])

NameError: name 'business_features' is not defined

In [ ]:
business_model.fit(data, fine_counts > 0);

In [ ]:
grader.score.ml__business_model(positive_probability(business_model))

## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [ ]:
# test_df = data[['CYCLE_1_SURVEY_DATE', 'CYCLE_2_SURVEY_DATE']]

In [56]:
# (pd.to_datetime(test_df['CYCLE_1_SURVEY_DATE']) - pd.to_datetime(test_df['CYCLE_2_SURVEY_DATE'])).apply(lambda x: x.days)

In [57]:
# test_df['delta']= test_df['CYCLE_1_SURVEY_DATE'] - test_df['CYCLE_2_SURVEY_DATE']

In [58]:
# test_df['delta'] = test_df['delta'].apply(lambda x: x.days)

In [59]:
# test_df.info()

In [60]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X=pd.DataFrame(X)
        results = (pd.to_datetime(X[self.t1_col])
                   - pd.to_datetime(X[self.t2_col])).apply(lambda x: x.days)
        results = results.values.reshape(-1,1)
        return results

In [61]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In [62]:
time_feature.fit_transform(data.head()).shape

(5, 1)

In the cell below we'll collect the cycle 1 survey features.

In [63]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, cycle_2_score.astype(int), test_size=0.2, random_state=42
)

In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as mse

fu = FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
])

gs = GridSearchCV(Lasso(max_iter=1000),
                  param_grid={'alpha': np.arange(0,3.5,0.5)},
                  cv=5,
                  n_jobs=4,
                  verbose=1)

survey_model = Pipeline([
    ('features', fu),
    ('poly', PolynomialFeatures()),
    ('decomp', TruncatedSVD(30)),
#     ('lr', RandomForestRegressor()),
#     ('lr', LinearRegression()),
#     ('lr', Lasso(alpha=3, max_iter=1000)),
    ('gs',gs)
])

NameError: name 'business_features' is not defined

In [66]:
survey_model.fit(X_train, y_train);

NameError: name 'survey_model' is not defined

In [ ]:
print('train:', survey_model.score(X_train, y_train))

In [ ]:
print('test:', survey_model.score(X_test, y_test))

In [ ]:
print('train:', mse(y_train, survey_model.predict(X_train)))
print('test:', mse(y_test, survey_model.predict(X_test)))

In [ ]:
survey_model.score(data, cycle_2_score.astype(int))

In [ ]:
grader.score.ml__survey_model(survey_model.predict)

*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*